# Test Code: Runs Recruitment Monitoring Bot

In [ ]:
email = 'sebastian.deri@gmail.com'
password = 'Sociologytemporary1!'
monitor_site(email, password, "Pilot", 20, 1, 60)

# Functions: Main and Helper

In [145]:
"""
Use monitor_site() to monitor TurkPrime recruitment.
Helper funtions: get_surveyid, add_recruit
"""

#load dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import random

In [12]:
#main function

def monitor_site(email, password, study, worlds, recruits):
    """
    The function monitor_site() has 5 parameters:
        1) email (TurkPrime email)
        2) password (TurkPrime password)
        3) study (root name of studies' HITs to monitor)
        4) worlds (number of experimental worlds)
        5) recruits (# total recruits per world, with 1:1 ratio per party),
        
    Running monitor_site('usr@exp.com', "mypw", 'Experiment', 20, 1) will
        1) log into TurkPrime with "usr@exp.com" and "mypw"
        2) monitor studies with 'Experiment' root name (e.g., Experiment1D),
        3) recruit 20 people to 1 experimental world (10 Dems, 10 Reps),
    """
    
    print("You are about to monitor a study with the following parameters:")
    print("{} worlds".format(worlds))
    print("{} recruit(s) per world".format(recruits))
    progress = str(input("Are the above parameters correct? Enter Y/N:")).lower()
    while True:
        if progress == 'y':
            print("Continuing on to monitor the recruitment website...")
            break
        else:
            raise RuntimeError("Breaking: check function parameters or input")

    #Create new Chrome session to access TurkPrime
    url_TP = "https://account.turkprime.com/Account/Login"
    driver = webdriver.Chrome("/usr/local/bin/chromedriver") #Mac location
    driver.implicitly_wait(10) #if load fails in 10s, give error
    driver.get(url_TP)
    time.sleep(5) #wait for TP login page load

    #Log into TP with username and pw
    try:
        print("Logging into TurkPrime...")
        inputRemem = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/form[1]/div[3]/div[3]/div[1]/label').click()
        inputEmail = driver.find_element_by_xpath('//*[@id="Email"]')
        inputEmail.send_keys(email)
        inputPass = driver.find_element_by_xpath('//*[@id="Password"]')
        inputPass.send_keys(password)
        inputPass.send_keys(Keys.ENTER)
        time.sleep(5) #wait after login
    except:
        raise RuntimeError("  Bot error: could not log in")

    #Navigate to Dashboard page and display 20 studies
    try:
        print("Navigating to Dashboard...")
        url_Dash = 'https://www.turkprime.com/LaunchedSurvey/Dashboard?page=1&filter=Live%2FPending&itemsPerPage=20'
        driver.get(url_Dash)
        time.sleep(5)
    except:
        raise RuntimeError("  Bot error: could not navigate to Dashboard")
        
    # ...
    pass






    #End monitoring
    print("\n\n\n\n")
    print("************************************************************")
    print("************************************************************")
    print("RECRUITMENT MONITORING ENDED")
    print("Please ensure TurkPrime is in good order before logging off.")
    closeBrowser = str(input("Do you wish to close the bot's browser? Enter Y/N: ")).lower()
    if closeBrowser == "y":
        print("Closing the bot's browser.")
        driver.quit()
    else:
        print("Keeping the bot's browser open.")
        pass
    print("\n\n")
    print("This recruitment monitoring program is now complete.")
    print("The recruitment website will no longer be monitored.")
    print("Goodbye!\n\n")

In [ ]:
#helper functions: get_surveyid, add_recruit

def get_surveyid(HITposition):
    HITposition = str(HITposition)
    print("Getting survey id for HIT {}...".format(HITposition))
    surveyid = driver.find_element_by_xpath(
            '//*[@id="surveyTableBody"]/tr[' +
            HITposition +
            ']/td[1]/div/ul/li[3]/span'
        )
    surveyid = surveyid.text[surveyid.text.index(":")+2:] #slice off id
    print("  Extracted survey id for HIT {}: {}".format(HITposition,surveyid))
    return surveyid


def add_recruit(HITposition):
    try:
        print("Editing HIT {}: addding recruit...".format(str(HITposition)))
        surveyid = get_surveyid(HITposition)
        url_HIT_edit = 'https://www.turkprime.com/LaunchedSurvey/EditPanelStudyToolkit/'+surveyid
        driver.get(url_HIT_edit)   
        time.sleep(5)
        #driver.find_element_by_xpath('//*[@id="page-top"]/div[6]/div/div[10]/button[1]').click() #edit warning
        #time.sleep(5)
    except:
        raise RuntimeError("  Bot error: could not edit HIT") 
    try:
        print("Updating HIT {} recruit count...".format(str(HITposition)))
        recruitsCurrent = driver.find_element_by_id("LaunchedSurveySettings_ParticipantsCount")
        recruitsCurrentN = recruitsCurrent.get_attribute("value")
        print("  HIT {} presently has {} recruits".format(str(HITposition),recruitsCurrentN))
        recruitsUpdatedN = str(int(recruitsCurrentN) + 1)
        recruitsCurrent.clear()
        recruitsCurrent.send_keys(recruitsUpdatedN)
        print("  HIT {} updated to have {} recruits".format(str(HITposition),recruitsUpdatedN))
        time.sleep(3)
    except:
        raise RuntimeError("  Bot error: could not add another recruit")
    try:
        print("Navigating to HIT {} save page...".format(str(HITposition)))
        driver.find_element_by_xpath('//*[@id="design-wizard-t-8"]').click()
        time.sleep(3)
    except:
        raise RuntimeError("  Bot error: could not navigate to save HIT page") 
    try:
        print("Saving HIT {}...".format(str(HITposition)))
        driver.find_element_by_xpath('//*[@id="submitBtn"]').click()
        time.sleep(3)
    except:
        raise RuntimeError("  Bot error: could not save HIT") 
    print("HIT {} sucessfully updated!".format(str(HITposition)))
    print("************************************************************\n")

# Class Constructor for Worlds' HITs and Party-based Groups

In [216]:
class hitGroup:
    """
    Class constructor: group of recruits with same HIT, world, and party.
    """
    
    global driver
    
    def __init__(self, groupName, groupIndex, completed, remaining):
        self.name      = groupName #project title for group name on TurkPrime
        self.index     = groupIndex #study group index position on Dashboard
        self.completed = completed #how many recruits completed the test so far
        self.remaining = remaining #how many recruits remain to be recruited
        self.root      = '//*[@id="surveyTableBody"]/tr['
        self.project   = root + str(groupIndex) + ']/td[1]/div/ul/li[1]/span/a'
        self.surveyID  = root + str(groupIndex) + ']/td[1]/div/ul/li[3]/span/span'
        self.progress  = root + str(groupIndex) + ']/td[3]/div[1]/div/p'
        self.launch    = root + str(groupIndex) + ']/td[4]/div/ul/li[1]/a'
        self.pause     = root + str(groupIndex) + ']/td[4]/div/ul/li[2]/a'
        self.resume    = root + str(groupIndex) + ']/td[4]/div/ul/li[2]/a'
        
    def check_surveyID(self):
        return self.name in driver.find_element_by_xpath(self.surveyID)
    
    def check_progress(self):
        return driver.find_element_by_xpath(self.progress)
            
    def __eq__(self, other):
        """Override the default equals behavior"""
        return self.completed == other.completed
    
    def __lt__(self, other):
        """Override the default less than behavior"""
        return self.completed < other.completed
    
    def __gt__(self, other):
        """Override the default greater than behavior"""
        return self.completed > other.completed
    
    def compare_progress(self, other):
        if self.completed == other.completed:
            return "Equal"
        elif self.completed > other.completed:
            print(self.name, "is ahead of", other.name, "by",
                str(self.completed - other.completed) + ":",
                self.completed, "to", other.completed)
            return self
        else:
            print(other.name, "is ahead of", self.name, "by",
                str(self.completed - other.completed) + ":", 
                other.completed, "to", self.completed)
            return other
        
    def click_launch(self):
        driver.find_element_by_xpath(self.launch).click()
    
    def click_pause(self):
        driver.find_element_by_xpath(self.pause).click()
    
    def click_resume(self):
        driver.find_element_by_xpath(self.resume).click()

    def recruit_completed(self, completed):
        #Increment completed count up and number remaining down
        self.completed += completed
        self.remaining -= completed
        
    def recruits_add(self, add):
        #Increase number of recruits remaining in group
        self.remaining += add

# Coding Sandbox: Functions to Test Next

In [142]:
#Create new Chrome session to access TurkPrime
url_TP = "https://account.turkprime.com/Account/Login"
driver = webdriver.Chrome("/usr/local/bin/chromedriver") #Mac location
driver.implicitly_wait(10) #if load fails in 10s, give error
driver.get(url_TP)
time.sleep(5) #wait for TP login page load

#Log into TP with username and pw
try:
    print("Logging into TurkPrime...")
    inputRemem = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/form[1]/div[3]/div[3]/div[1]/label').click()
    inputEmail = driver.find_element_by_xpath('//*[@id="Email"]')
    inputEmail.send_keys(email)
    inputPass = driver.find_element_by_xpath('//*[@id="Password"]')
    inputPass.send_keys(password)
    inputPass.send_keys(Keys.ENTER)
    time.sleep(5) #wait after login
except:
    raise RuntimeError("  Bot error: could not log in")

#Navigate to Dashboard page and display 20 studies
try:
    print("Navigating to Dashboard...")
    url_Dash = 'https://www.turkprime.com/LaunchedSurvey/Dashboard?page=1&filter=Live%2FPending&itemsPerPage=20'
    driver.get(url_Dash)
    time.sleep(5)
except:
    raise RuntimeError("  Bot error: could not navigate to Dashboard")




# Construct group objects


Logging into TurkPrime...
Navigating to Dashboard...
Editing HIT 1: addding recruit...
Getting survey id for HIT 1...
  Extracted survey id for HIT 1: 104873
Updating HIT recruit count...
  HIT presently has 9 recruits
  HIT updated to have 10 recruits
Navigating to save HIT page...
Saving HIT...


# Untested Code

In [ ]:
worlds = 1 # <-- DELETE THIS
recruits = 20 # <-- DELETE THIS


#Initialize Party-classes
print("Initializing worlds' party-classes for {} worlds...".format(str(worlds)))
index = 1 #initialize worlds' party-classes index position on Dashboard website
for world in range(1,worlds+1):
    globals()['ExperimentD{}'.format(world)] = hitGroup('ExperimentD'+str(world), index, 0, 10)
    index += 2 #increment index to odds to match Dashboard study indexing
    globals()['ExperimentR{}'.format(world)] = hitGroup('ExperimentR'+str(world), index, 0, 10)
    index += 2

#Monitor recruitment from Dashboard
print("Begin monitoring recruitment website from Dashboard...") #TODO: include timer, counter, etc. to check still running
recruit = None #TODO: sum progress cells
while recruit <= recruits:
    pass

    recruit += 1

    


In [217]:
index = 1
for world in range(1,11):
    globals()['ExperimentD{}'.format(world)] = hitGroup('ExperimentD'+str(world), index, 0, 10)
    index += 2
    globals()['ExperimentR{}'.format(world)] = hitGroup('ExperimentR'+str(world), index, 0, 10)
    index += 2

ExperimentD1.recruit_completed(1)
print(vars(ExperimentD1),'\n')
print(vars(ExperimentR1),'\n')
ExperimentD1.compare_progress(ExperimentR1)

{'name': 'ExperimentD1', 'index': 1, 'completed': 1, 'remaining': 9, 'root': '//*[@id="surveyTableBody"]/tr[', 'project': '//*[@id="surveyTableBody"]/tr[1]/td[1]/div/ul/li[1]/span/a', 'surveyID': '//*[@id="surveyTableBody"]/tr[1]/td[1]/div/ul/li[3]/span/span', 'progress': '//*[@id="surveyTableBody"]/tr[1]/td[3]/div[1]/div/p', 'launch': '//*[@id="surveyTableBody"]/tr[1]/td[4]/div/ul/li[1]/a', 'pause': '//*[@id="surveyTableBody"]/tr[1]/td[4]/div/ul/li[2]/a', 'resume': '//*[@id="surveyTableBody"]/tr[1]/td[4]/div/ul/li[2]/a'} 

{'name': 'ExperimentR1', 'index': 3, 'completed': 0, 'remaining': 10, 'root': '//*[@id="surveyTableBody"]/tr[', 'project': '//*[@id="surveyTableBody"]/tr[3]/td[1]/div/ul/li[1]/span/a', 'surveyID': '//*[@id="surveyTableBody"]/tr[3]/td[1]/div/ul/li[3]/span/span', 'progress': '//*[@id="surveyTableBody"]/tr[3]/td[3]/div[1]/div/p', 'launch': '//*[@id="surveyTableBody"]/tr[3]/td[4]/div/ul/li[1]/a', 'pause': '//*[@id="surveyTableBody"]/tr[3]/td[4]/div/ul/li[2]/a', 'resume'